In [ ]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320.4 MB 24 kB/s 
     |████████████████████████████████| 52 kB 763 kB/s 
     |████████████████████████████████| 2.9 MB 44.3 MB/s 
     |████████████████████████████████| 20.1 MB 11.3 MB/s 
     |████████████████████████████████| 459 kB 57.0 MB/s 
     |████████████████████████████████| 1.6 MB 35.7 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing ins

In [ ]:
!pip install gym[atari]
!pip install autorom[accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=2dc57862e27cbc890ceefa50405dac88f15c86170adde7120acc52eb68f32a65
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built AutoROM.accept-rom-license


In [ ]:
import gym
import matplotlib.pyplot as plt
import random
import time

In [ ]:
from ale_py import ALEInterface
ale = ALEInterface()

from ale_py.roms import AirRaid
ale.loadROM(AirRaid)

In [ ]:
env = gym.make("ALE/AirRaid-v5", render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

RuntimeError: ignored

In [ ]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

TypeError: ignored

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model

In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 61, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 29, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 27, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               42467840  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Training for 10000 steps ...


/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:228: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  "Core environment is written in old step API which returns one bool instead of two. "


  420/10000: episode: 1, duration: 23.461s, episode steps: 420, steps per second:  18, episode reward: 475.000, mean reward:  1.131 [ 0.000, 100.000], mean action: 2.662 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  660/10000: episode: 2, duration: 12.669s, episode steps: 240, steps per second:  19, episode reward: 225.000, mean reward:  0.938 [ 0.000, 75.000], mean action: 2.587 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  859/10000: episode: 3, duration: 10.107s, episode steps: 199, steps per second:  20, episode reward: 125.000, mean reward:  0.628 [ 0.000, 75.000], mean action: 2.246 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1416/10000: episode: 4, duration: 1058.587s, episode steps: 557, steps per second:   1, episode reward: 575.000, mean reward:  1.032 [ 0.000, 100.000], mean action: 2.461 [0.000, 5.000],  loss: 75.320372, mean_q: 24.595057, mean_eps: 0.891280
 1899/10000: episode: 5, duration: 1217.389s, episode steps: 483, steps per second:   0, epi

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


/usr/local/lib/python3.7/dist-packages/gym/core.py:44: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  "The argument mode in render method is deprecated; "


Error: ignored